<a href="https://colab.research.google.com/github/hammedb197/GiveLIVE_AI/blob/master/emotional_detection_with_data_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!cat 'dataset.csv'

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('dataset.csv')

In [6]:
data.head()

,label,text,Unnamed: 2
0,joy,On days when I feel close to my partner and ot...,NaN
1,fear,Every time I imagine that someone I love or I ...,NaN
2,anger,When I had been obviously unjustly treated and...,NaN
3,sadness,When I think about the short time that we live...,NaN
4,disgust,At a gathering I found myself involuntarily si...,NaN


In [7]:
data.label.value_counts()

joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: label, dtype: int64

In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
stopwords = stopwords.words('english')

In [0]:

def clean_data(data): 
  clean_data = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT)|(http[:/.A-Za-z0-9]+)|(\W)|([0-9])", " ", data)
  token_data = word_tokenize(clean_data)
  lemma_data = [lemma.lemmatize(i) for i in token_data]
  stopword_data = ' '.join([i for i in lemma_data if i not in stopwords])
  return stopword_data

# (@)|(http[:/.A-Za-z0-9]+)|(RT)|(#)|(\W)|([0-9])


In [0]:
data['text'] = data['text'].apply(clean_data)

In [14]:
data['label'].isnull().sum()

0

In [0]:
data['label'] = "__label__" + data['label']

labels start by the __label__ prefix

In [16]:
data.head()

,label,text,Unnamed: 2
0,__label__joy,On day I feel close partner friend When I feel...,NaN
1,__label__fear,Every time I imagine someone I love I could co...,NaN
2,__label__anger,When I obviously unjustly treated possibility ...,NaN
3,__label__sadness,When I think short time live relate period lif...,NaN
4,__label__disgust,At gathering I found involuntarily sitting nex...,NaN


In [0]:
data = data.iloc[:,  :2]

In [18]:
data.head()

,label,text
0,__label__joy,On day I feel close partner friend When I feel...
1,__label__fear,Every time I imagine someone I love I could co...
2,__label__anger,When I obviously unjustly treated possibility ...
3,__label__sadness,When I think short time live relate period lif...
4,__label__disgust,At gathering I found involuntarily sitting nex...


In [34]:
data.shape

(7516, 2)

 full dataset contains 7516 examples, 5637 train set, 1879 tst set 
 

---



In [35]:
75/100 * 7516

5637.0

In [89]:
7516 - 5637

1879

In [0]:
data.to_csv(r'data.txt', header=None, index=None, sep=' ', mode='a')

In [21]:
!wc data.txt

  7516 103653 692855 data.txt


In [0]:
!head -n 5637 data.txt > data.train
!tail -n 1879 data.txt > data.valid

In [26]:
!pip install fasttext

  Using cached https://files.pythonhosted.org/packages/f8/85/e2b368ab6d3528827b147fdb814f8189acc981a4bc2f99ab894650e05c40/fasttext-0.9.2.tar.gz
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016139 sha256=0879c4611db613eb6d32a4615a220b10fe9e2f5ddd2d675830575c6795a2ce1b
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/usr/local/lib/python3.6/dist-packages/fasttext-0.9.2.dist-info/INSTALLER.pip'



In [0]:
import fasttext

In [0]:
model = fasttext.train_supervised(input="data.train", epoch=25, wordNgrams=2)

In [0]:
model.save_model('model.bin')

In [0]:
predict = model.predict('I am happy today')

In [36]:
model.test("data.valid")

(1879, 0.5705162320383182, 0.5705162320383182)

In [58]:
predict[0][0].split("__label__")[1]

'joy'

Test model

In [44]:
model = fasttext.load_model('/content/model.bin')

In [0]:
def predict_emotion(data):
  clean_data = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(RT)|(http[:/.A-Za-z0-9]+)|(\W)|([0-9])", " ", data)
  token_data = word_tokenize(clean_data)
  lemma_data = [lemma.lemmatize(i) for i in token_data]
  stopword_data = ' '.join([i for i in lemma_data if i not in stopwords])
  predict = model.predict(stopword_data)
  output = predict[0][0].split("__label__")[1]
  return output


In [56]:
predict_emotion("basically, I am the only one with an ugly product flow on this teamYawning face.")

'guilt'